In [1]:
COLAB: bool = True
if COLAB:
  !rm -rdf 6GSmartRRM
  !rm -rdf /content/g6smart
  !git clone https://github.com/RubenCid35/6GSmartRRM
  !mv 6GSmartRRM/* /content/
  !pip install -e .
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

Cloning into '6GSmartRRM'...
remote: Enumerating objects: 255, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 255 (delta 133), reused 185 (delta 72), pack-reused 0 (from 0)
Receiving objects: 100% (255/255), 1020.85 KiB | 10.42 MiB/s, done.
Resolving deltas: 100% (133/133), done.
mv: cannot move '6GSmartRRM/data' to '/content/data': Directory not empty
mv: cannot move '6GSmartRRM/docs' to '/content/docs': Directory not empty
mv: cannot move '6GSmartRRM/notebooks' to '/content/notebooks': Directory not empty
mv: cannot move '6GSmartRRM/static' to '/content/static': Directory not empty
Obtaining file:///content
  Preparing metadata (setup.py) ... done
  Attempting uninstall: g6smart
    Found existing installation: g6smart 0.1
    Uninstalling g6smart-0.1:
      Successfully uninstalled g6smart-0.1
  Running setup.py develop for g6smart
Mounted at /content/drive


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# simple data manipulation
import numpy  as np
import pandas as pd

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lrs
from   torch.utils.data import DataLoader, TensorDataset, random_split

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from collections import defaultdict

# progress bar
from   tqdm.notebook import tqdm, trange
import wandb
# remove warnings (remove deprecated warnings)
import warnings
warnings.simplefilter('ignore')

# visualization of resultsa
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from   matplotlib.ticker import MaxNLocator
import seaborn           as sns
import plotly.express as px


# wheter we are using colab or not
import os
if not COLAB and not os.path.exists('./data/simulations'):
    os.chdir('..')
    print("current path: ", os.getcwd())

# Simulation Settings
from g6smart.sim_config import SimConfig
from g6smart.evaluation import rate as rate
from g6smart.evaluation import rate_torch as rate_metrics
from g6smart.proposals  import loss as loss_funcs, rate_cnn, rate_dnn
from g6smart.data import load_data, create_datasets, download_simulations_data
from g6smart.track import setup_wandb, real_time_plot

config = SimConfig(0)
print(config)

wandb: Currently logged in as: rubencid001 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Simulation Parameters: 

|                      name |                     value |
---------------------------------------------------------
|        num_of_subnetworks |                   20.0000 |
|              n_subchannel |                    4.0000 |
|             deploy_length |                   20.0000 |
|             subnet_radius |                    1.0000 |
|                      minD |                    0.8000 |
|               minDistance |                    2.0000 |
|                 bandwidth |             40000000.0000 |
|              ch_bandwidth |             10000000.0000 |
|                        fc |           6000000000.0000 |
|                    lambdA |                    0.0500 |
|                  clutType |                     dense |
|                  clutSize |                    2.0000 |
|                  clutDens |                    0.6000 |
|                   shadStd |                    7.2000 |
|                 max_power |                  

## Datasets

In [4]:
simulation_path, models_path = download_simulations_data(COLAB)
print("simulations data paths:", simulation_path)
print("saved model location  :", models_path)


failed to mount drive, cause: [Errno 125] Operation canceled: 'MyDrive'
 ----> simulations-200K-sisa.zip
simulations data paths: /content/simulations
saved model location  : /content/drive/MyDrive/TFM/models/


In [5]:
csi_data = load_data(simulation_path, n_samples=200_000)
train_loader, valid_loader, tests_loader = create_datasets(
    csi_data, split_sizes=[130_000, 60_000, 10_000], batch_size=256, seed=101
)

## FNN

## FNN Approach

In [6]:
def min_approx(x: torch.Tensor, p: float = 1e8, mu: float = 0.):
    """
    Differentiable Approximation of Minimum Function. This function approximates
    the value of min(x)

      # based on fC https://mathoverflow.net/questions/35191/a-differentiable-approximation-to-the-minimum-function
    """
    return mu - (1 / p) * torch.logsumexp(-p * (x - mu), dim = 1)

def loss_pure_rate(config: SimConfig, C: torch.Tensor, A: torch.Tensor, mode: str = 'sum', p: int = 1e8) -> torch.Tensor:
    sinr = rate_metrics.signal_interference_ratio(config, C, A, None)
    mask = torch.sigmoid(10 * (sinr - 0.01))
    rate = torch.sum(torch.log2(1 + sinr) * A * mask, dim = 1)
    rate = rate / torch.sum(A, dim = 1)

    if mode == 'sum':
      loss_rate = torch.sum(rate, dim = 1)
    elif mode == 'min':
      loss_rate = min_approx(rate, p)
    elif mode == 'max':
      loss_rate = torch.sum(rate, dim = 1)
    elif mode == "mean":
      loss_rate = torch.mean(rate, dim = 1)
    return - loss_rate

def loss_interference(C: torch.Tensor, A: torch.Tensor):
  losses = A.unsqueeze(-1) * 10 * torch.log10(C + 1e-12) * A.unsqueeze(-2)
  # losses = 10 * torch.log10(losses + 1e-12)
  return torch.sum(losses.flatten(start_dim = 1), dim = 1)


In [7]:
sample = torch.rand(256, 100000)
mint = torch.max(sample, dim = 1)[0]
for p in range(-2, 10):
  min1 = loss_funcs.min_approx(sample, p = - 10 ** p)
  min2 = min_approx(sample, p = - 10 ** p)
  dis1 = torch.abs(min1 - mint).mean().item()
  dis2 = torch.abs(min2 - mint).mean().item()
  print(f"{10 ** p:3.2e} -> min (lib): {dis1:7.6e}\tmin (new): {dis2:7.6e}")

1.00e-02 -> min (lib): 1.150793e+03	min (new): 1.150793e+03
1.00e-01 -> min (lib): 1.146334e+02	min (new): 1.146334e+02
1.00e+00 -> min (lib): 1.105427e+01	min (new): 1.105427e+01
1.00e+01 -> min (lib): 9.210697e-01	min (new): 9.210697e-01
1.00e+02 -> min (lib): 6.910025e-02	min (new): 6.910025e-02
1.00e+03 -> min (lib): 4.610383e-03	min (new): 4.610383e-03
1.00e+04 -> min (lib): 2.353974e-04	min (new): 2.353974e-04
1.00e+05 -> min (lib): 6.121816e-06	min (new): 6.121816e-06
1.00e+06 -> min (lib): 7.962808e-08	min (new): 7.962808e-08
1.00e+07 -> min (lib): 2.491288e-08	min (new): 2.491288e-08
1.00e+08 -> min (lib): 1.303852e-08	min (new): 1.303852e-08
1.00e+09 -> min (lib): 2.817251e-08	min (new): 2.817251e-08


In [8]:
class FNNModel(nn.Module):
  def __init__(
      self, N: int, K: int,
      hidden_dim: int = 1024, hidden_layers: int = 4,
      dropout: float = 1e-1, to_matrix: bool = False
  ):
    super().__init__()
    self.N = N
    self.K = K

    self.triu_mask = torch.triu(
        torch.ones(self.N, self.N, dtype = torch.bool),
        diagonal = 1
    )
    self.to_matrix = to_matrix
    if self.to_matrix:
      self.in_size  = self.N * (self.N - 1) // 2
    else:
      self.in_size  = self.N * (self.N - 1) // 2 * self.K

    self.out_size = self.K * self.N

    layers = [ ]
    #nn.BatchNorm1d(self.in_size),
    #nn.BatchNorm2d(self.K), nn.Flatten()

    dims = [self.in_size] + [hidden_dim] * (hidden_layers + 1) + [self.out_size]

    for _in, _out in zip(dims[:-1], dims[1:]):
      layers.append(nn.Linear(_in, _out))
      torch.nn.init.kaiming_normal_(layers[-1].weight, nonlinearity='relu')
      layers.append(nn.ReLU())
      layers.append(nn.BatchNorm1d(_out))
      layers.append(nn.Dropout(dropout))

    self.model = nn.Sequential(*layers[:-3])

  def preprocess(self, H: torch.Tensor) -> torch.Tensor:
    shape = 'x'.join(map(str, H.shape[1:]))
    assert len(H.shape)  == 4 and H.size(1) == self.K and H.size(2) == self.N and H.size(3) == self.N, (
        f"The input tensor needs to match Bx{self.K}x{self.N}x{self.N}, not Bx{shape} (B is the batch size)"
    )

    # take only the upper triangle
    if self.to_matrix:
      H = torch.sum(H, dim = 1)
      H = H[:, self.triu_mask]
    else:
      Hd = torch.diagonal(H, dim1=2, dim2=3).unsqueeze(-1)
      H  = H / Hd
      H  = H[:, :, self.triu_mask] # B x K x (N * (N - 1) // 2)


    # reshape
    H = 10 * torch.log10(H + 1e-12)

    # could this be learned
    mean = torch.mean(H, dim = 1, keepdim=True)
    std  = torch.std( H, dim = 1, keepdim=True)
    H    = (H - mean) / (std + 1e-8)
    return H

  def forward(self, H: torch.Tensor) -> torch.Tensor:
    H = self.preprocess(H)
    x = self.model(H)
    x = x.reshape(-1, self.K, self.N)
    return F.softmax(x, dim = 1)


In [9]:
torch.autograd.set_detect_anomaly(True)
print(torch.cuda.memory_summary(device=None, abbreviated=True))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Requested memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------

In [17]:
# Example

model = FNNModel(20, 4, 512, 2, 0.1, True) # .to(device)
#model = rate_cnn.RateConfirmAllocCNNModel(20, 4, 0.1, True)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

losses = []
for step in range(30):
  model.train()
  total_loss = 0.
  total_bin_error = 0.
  for sample in tqdm(train_loader, desc = "training: ", unit=" batch", total = len(train_loader), leave = False):
    optimizer.zero_grad()

    sample = sample[0].to(device)
    alloc_prob = model(sample)        # soft output
    loss = loss_pure_rate(config, sample, alloc_prob, 'min', p = 1e6).mean()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
    optimizer.step()
    total_loss += loss.item()
    total_bin_error += loss_funcs.binarization_error(alloc_prob)

    del sample, alloc_prob, loss
    torch.cuda.empty_cache()

  training_loss = total_loss / len(train_loader)
  training_bin_loss = total_bin_error / len(train_loader)

  model.eval()
  total_loss = 0.
  total_bin_error = 0.

  with torch.no_grad():
    for sample in tqdm(valid_loader, desc = "validation: ", unit=" batch", total = len(valid_loader), leave = False):
      sample = sample[0].to(device)
      alloc_prob = model(sample)        # soft output
      loss = loss_pure_rate(config, sample, alloc_prob,  'min', p = 1e6).mean()
      # loss = loss_interference(sample, alloc_prob).mean()
      total_loss += loss.item()
      total_bin_error += loss_funcs.binarization_error(alloc_prob)

      del sample, alloc_prob, loss
      torch.cuda.empty_cache()


  total_loss = total_loss / len(valid_loader)
  total_bin_error = total_bin_error / len(valid_loader)
  print(f"[{step:>3d}] training loss: {training_loss:7.4f} (bin error: {training_bin_loss:6.4f})", end = "\t")
  print(f"validation loss: {total_loss:7.4f}  (bin error: {total_bin_error:6.4f})")





training:   0%|          | 0/508 [00:00<?, ? batch/s]

validation:   0%|          | 0/235 [00:00<?, ? batch/s]

[  0] training loss: -0.7054 (bin error: 0.1818)	validation loss: -0.7165  (bin error: 0.1962)


training:   0%|          | 0/508 [00:00<?, ? batch/s]

validation:   0%|          | 0/235 [00:00<?, ? batch/s]

...

validation:   0%|          | 0/235 [00:00<?, ? batch/s]

[ 28] training loss: -0.9215 (bin error: 0.1485)	validation loss: -0.7146  (bin error: 0.1595)


training:   0%|          | 0/508 [00:00<?, ? batch/s]

validation:   0%|          | 0/235 [00:00<?, ? batch/s]

[ 29] training loss: -0.9259 (bin error: 0.1467)	validation loss: -0.7134  (bin error: 0.1577)


In [18]:
import json
model.eval()
total_loss = 0.
total_bin_error = 0.
metrics = defaultdict(lambda : 0)
with torch.no_grad():
  for sample in tqdm(tests_loader, desc = "testing: ", unit=" batch", total = len(tests_loader), leave = False):
    sample = sample[0].to(device)
    alloc_prob = model(sample)        # soft output
    loss = loss_pure_rate(config, sample, alloc_prob,  'min', p = 1e6).mean()
    # loss = loss_interference(sample, alloc_prob).mean()
    total_loss += loss.item()
    total_bin_error += loss_funcs.binarization_error(alloc_prob)

    metrics = loss_funcs.update_metrics(metrics, alloc_prob, sample, None, config, 4)

    del sample, alloc_prob, loss
    torch.cuda.empty_cache()

total_loss = total_loss / len(tests_loader)
total_bin_error = total_bin_error / len(tests_loader)

metrics = { key: val / len(tests_loader) for key, val in metrics.items()}

print("testing run:")
print("testing batches: ", len(tests_loader))
print("test test error: ", total_loss)
print("test test binarization error: ", total_bin_error)
print("bit rate / quality metrics:\n", json.dumps(metrics, indent = 2))


testing:   0%|          | 0/40 [00:00<?, ? batch/s]

TypeError: FNNModel.forward() takes 2 positional arguments but 3 were given